In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit
from IPython.display import Image

In [ ]:
N = 1000
LIM = (-5, 5)
rs = np.random.RandomState(0)
X = rs.uniform(LIM[0], LIM[1], size=(N, 2))
prob = expit(X[..., 0] * X[..., 1])
prob = expit(X[..., 0])
y = ((rs.uniform(size=N) < prob) * 2 - 1)

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(X[..., 0], X[..., 1], c=y)
plt.xlim(LIM); plt.ylim(LIM);

In [ ]:
def evaluate_grid(cl, xlim=LIM, ylim=LIM):
    xx0, xx1 = np.meshgrid(np.linspace(xlim[0], xlim[1], 100), np.linspace(ylim[0], ylim[1], 100))
    XXflat = np.array([xx0.ravel(), xx1.ravel()]).T
    if hasattr(cl, 'predict_proba'):
        yyflat = cl.predict_proba(XXflat)[..., 1]
    elif hasattr(cl, 'decision_function'):
        yyflat = cl.decision_function(XXflat)
    else:
        yyflat = cl.predict(XXflat)

    return yyflat.reshape(xx0.shape)

In [ ]:
from sklearn import tree, svm, naive_bayes, neighbors, ensemble

cls = []

if 1:
    cl_dt = tree.DecisionTreeClassifier(max_depth=8)
    #cl_dt = ensemble.RandomForestClassifier(n_estimators=100, max_depth=8, criterion='entropy')
    cl_dt.fit(X, y)
    cls.append(cl_dt)

if 1:
    cl_lsvm = svm.LinearSVC()
    cl_lsvm.fit(X, y)
    cls.append(cl_lsvm)

if 1:
    cl_ksvm = svm.SVC(C=0.00000001)
    cl_ksvm.fit(X, y)
    cls.append(cl_ksvm)
    
if 0:
    cl_nb = naive_bayes.GaussianNB()
    cl_nb.fit(X, y)
    cls.append(cl_nb)

if 1:
    cl_knn = neighbors.KNeighborsClassifier(n_neighbors=41)
    cl_knn.fit(X, y)
    cls.append(cl_knn)
    
plt.figure(figsize=(6*len(cls), 6)) 
xx0, xx1 = np.meshgrid(np.linspace(LIM[0], LIM[1], 100), np.linspace(LIM[0], LIM[1], 100))
for i, cl in enumerate(cls):
    plt.subplot(1, len(cls), 1+i)
    plt.contourf(xx0, xx1, evaluate_grid(cl), cmap=plt.cm.RdBu_r)
    plt.scatter(X[..., 0], X[..., 1], c=y)
    plt.xlim(LIM); plt.ylim(LIM)

In [ ]:
# Requires the probram Graphviz to be installed
tree.export_graphviz(cl_dt, out_file='tree.dot')
!dot tree.dot -T png -o tree.png
Image('tree.png')